In [34]:
%load_ext autoreload

%autoreload 
from getdata import YahooFinance
from getdata import SQLrepo
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.ar_model import AutoReg
from dateutil.parser import parse
from model_s import StockModel
from datetime import datetime
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
import plotly.express as px



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
model=StockModel(table_name="Google", order=(6,0,0))
model.train_model(cut=0.95,returns= True)

Starting Google training Arima(6, 0, 0) with Returns True
Performing Walk Forward Val.. for 11 OBS with trained==> 205
Null Values Check ====> 0


100%|██████████| 11/11 [00:02<00:00,  4.10it/s]


=========== Google Training Ends ===========
Test MAE ==> 0.011135818367462936
Training Baseline==> 0.010131833297524621


In [36]:

#model.plot_pred
model.trained_model.forecast(2)

2021-10-18    0.003592
2021-10-19    0.003774
Freq: B, Name: predicted_mean, dtype: float64

In [37]:
model.y_test

Date
2021-10-04   -0.019767
2021-10-05    0.018032
2021-10-06    0.008643
2021-10-07    0.013334
2021-10-08    0.006254
2021-10-11   -0.008629
2021-10-12   -0.015373
2021-10-13    0.008682
2021-10-14    0.025468
2021-10-15    0.001860
2021-10-18    0.009074
Freq: B, Name: Close, dtype: float64

In [38]:
model.preds_actual

,y_test,y_pred
2021-10-04,-0.019767,0.004259
2021-10-05,0.018032,0.012483
2021-10-06,0.008643,0.001399
2021-10-07,0.013334,0.010817
2021-10-08,0.006254,-0.005455
2021-10-11,-0.008629,0.007680
2021-10-12,-0.015373,-0.000631
2021-10-13,0.008682,0.000164
2021-10-14,0.025468,0.000670
2021-10-15,0.001860,0.003460


# Strategy: 1 

In [74]:
y_pred=model.preds
y_prices=model.prices

In [83]:
import datetime

threshold = 0.005
portfolio = []  # To store the portfolio of trades
investment = 1000  # Initial investment amount
portfolio_value = investment  # Portfolio value starts with the initial investment

# Iterate through the predictions and prices
for i in range(len(y_pred)-1):  # Subtract 1 to avoid index out of range error on the last day
    if y_pred[i] > threshold:
        # Buy the stock
        buy_price = y_prices[i+1]
        shares_to_buy = portfolio_value / buy_price  # Calculate the number of shares to buy
        date = y_pred.index[i].strftime("%Y-%m-%d")  # Get the date from the index
        reason = f"The prediction on {date} were above threshold "  # Change this to the specific reason for buying
        portfolio.append((f'Buy on {y_pred.index[i+1]}', buy_price, date, shares_to_buy, reason))
        portfolio_value -= investment  # Deduct the investment amount from the portfolio value

        # Sell the stock on the next trading day
        sell_price = y_prices[i+2]
        shares_to_sell = shares_to_buy
        date = y_pred.index[i+2].strftime("%Y-%m-%d")  # Get the date from the next index
        portfolio.append(('Sell', sell_price, date, shares_to_sell, ""))  # Add an empty string as a placeholder
        portfolio_value += shares_to_sell * sell_price  # Add the sale proceeds to the portfolio value

# Display the portfolio
for trade in portfolio:
    action, price, date, shares, reason = trade[:5]  # Extract the first 5 elements
    print(f"{date} - {action} {shares:.2f} shares at price {price} - Reason: {reason}")

# Calculate the final portfolio value
final_portfolio_value = portfolio_value

print("Initial Investment:", investment)
print("Final Portfolio Value:", final_portfolio_value)


2022-12-06 - Buy on 2022-12-07 00:00:00 10.51 shares at price 95.1500015258789 - Reason: The prediction on 2022-12-06 were above threshold 
2022-12-08 - Sell 10.51 shares at price 93.94999694824219 - Reason: 
2023-02-08 - Buy on 2023-02-09 00:00:00 10.34 shares at price 95.45999908447266 - Reason: The prediction on 2023-02-08 were above threshold 
2023-02-10 - Sell 10.34 shares at price 94.86000061035156 - Reason: 
2023-02-13 - Buy on 2023-02-14 00:00:00 10.20 shares at price 94.94999694824219 - Reason: The prediction on 2023-02-13 were above threshold 
2023-02-15 - Sell 10.20 shares at price 97.0999984741211 - Reason: 
Initial Investment: 1000
Final Portfolio Value: 959.0728437308283


In [81]:
import datetime

threshold = 0.005
portfolio = []  # To store the portfolio of trades
investment = 1000  # Initial investment amount
portfolio_value = investment  # Portfolio value starts with the initial investment

# Iterate through the predictions and prices
for i in range(len(y_pred)-1):  # Subtract 1 to avoid index out of range error on the last day
    if y_pred[i] > threshold:
        # Buy the stock
        buy_price = y_prices[i+1]
        shares_to_buy = portfolio_value / buy_price  # Calculate the number of shares to buy
        date = y_pred.index[i].strftime("%Y-%m-%d")  # Get the date from the index
        reason = f"The prediction on {date} were above threshold "  # Change this to the specific reason for buying
        portfolio.append((f'Buy on {y_pred.index[i+1]}', buy_price, date, shares_to_buy, reason))
        portfolio_value -= investment  # Deduct the investment amount from the portfolio value

        # Sell the stock on the next trading day if it is within the bounds of the price array
        if i + 2 < len(y_prices):
            sell_price = y_prices[i+2]
            shares_to_sell = shares_to_buy
            date = y_pred.index[i+2].strftime("%Y-%m-%d")  # Get the date from the next index
            portfolio.append(('Sell', sell_price, date, shares_to_sell, ""))  # Add an empty string as a placeholder
            portfolio_value += shares_to_sell * sell_price  # Add the sale proceeds to the portfolio value

# Display the portfolio
for trade in portfolio:
    action, price, date, shares, reason = trade[:5]  # Extract the first 5 elements
    print(f"{date} - {action} {shares:.2f} shares at price {price} - Reason: {reason}")

# Calculate the final portfolio value
final_portfolio_value = portfolio_value

print("Initial Investment:", investment)
print("Final Portfolio Value:", final_portfolio_value)


2022-12-06 - Buy on 2022-12-07 00:00:00 10.51 shares at price 95.1500015258789 - Reason: The prediction on 2022-12-06 were above threshold 
2022-12-08 - Sell 10.51 shares at price 93.94999694824219 - Reason: 
2023-02-08 - Buy on 2023-02-09 00:00:00 10.34 shares at price 95.45999908447266 - Reason: The prediction on 2023-02-08 were above threshold 
2023-02-10 - Sell 10.34 shares at price 94.86000061035156 - Reason: 
2023-02-13 - Buy on 2023-02-14 00:00:00 10.20 shares at price 94.94999694824219 - Reason: The prediction on 2023-02-13 were above threshold 
2023-02-15 - Sell 10.20 shares at price 97.0999984741211 - Reason: 
Initial Investment: 1000
Final Portfolio Value: 959.0728437308283
